In [ ]:
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
matplotlib.use('TkAgg')

import warnings
warnings.filterwarnings("ignore")

from glob import glob
from dea_tools.temporal import temporal_statistics, xr_phenology
from dea_tools.spatial import xr_rasterize
from utils import calculate_indices, open_ncs
import hvplot.pandas
import hvplot.xarray

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

import datetime as dt



%matplotlib inline

Começando pela fazenda da MaVi

-18.07179°,-50.79121°

In [ ]:
parcels = gpd.read_file('/mnt/geodata/Clientes/Mavi/limites_formosa_do_rio_preto.gpkg', layer='talhoes')
farm = gpd.read_file('/mnt/geodata/Clientes/Mavi/limites_formosa_do_rio_preto.gpkg')
folder = '/mnt/geodata/Clientes/0FARMS/9a151331-1b50-4804-b416-c217d45d6ac6/'
files = sorted(glob(f'{folder}/L*.nc'))
files

In [ ]:
#parcels = gpd.read_file('/mnt/geodata/Clientes/Mavi/limites_formosa_do_rio_preto.gpkg', layer='talhoes')
farmgroupid = '2a80d89c-659a-4122-95a8-6a38288f879a'
farm = gpd.read_file('/mnt/geodata/Clientes/x_MT_lookslikenewfields/fewfarms.gpkg')
parcels = farm
folder = f'/mnt/geodata/Clientes/0FARMS/{farmgroupid}/'
files = sorted(glob(f'{folder}/L*.nc'))
files

In [ ]:
ds = open_ncs(folder,'Lan')
ds.dims

In [ ]:
ds1 = ds.resample(time='W').max(skipna=True)
ds1.dims

In [ ]:
ds1['T']

In [ ]:
vi = 'LAI'
vis = calculate_indices( ds1 , index = [vi], drop = True, collection = 'Landsat')
#vis['kNDVI'] = xr.where(vis['kNDVI'] > 0.4, np.nan, vis['kNDVI'])
# vis['NDVI'] = xr.where((vis['NDVI'] > 0.74) | (vis['NDVI'] < -0.05), np.nan, vis['NDVI'])
vis[vi] = xr.where((vis[vi] > 7.4) | (vis[vi] < 1), np.nan, vis[vi])


### filtering, gap-filling, smoothening,
all that is need to have a nice dataset for **phenology** 

- I've tested smoothening window and 3 and 5 worked well. 

In [ ]:
# xr.corr(vis['LAI'],vis['NDVI'], dim='time').values.mean()
days = 80
window = 5
vis_f1 = vis[vi].interpolate_na( dim = 'time' , 
                                method =  'pchip', # pchip , spline
                                max_gap = np.timedelta64( days , 'D' )
                                ).rolling(time=window).mean(skipna=True)

In [ ]:
save = True
if save:
    days = 50
    #vis.to_netcdf(f'{folder}/vis_raw.nc')
    for window in [1,3,5]:
        vis_f1 = vis.interpolate_na( dim = 'time' , 
                                    method =  'pchip', # pchip , spline
                                        max_gap = np.timedelta64( days , 'D' )
                                        ).rolling(time=window).mean(skipna=True)
        vis_f1.to_netcdf(f'{folder}/vis_mgdays_{days}_window_{window}.nc')

### Phenology
The metrics:

            SOS = Date of start of season
            POS = Date of peak of season
            EOS = Date of end of season
            vSOS = Value at start of season
            vPOS = Value at peak of season
            vEOS = Value at end of season
            Trough = Minimum value of season
            LOS = Length of season (Days)
            AOS = Amplitude of season (in value units)
            ROG = Rate of greening
            ROS = Rate of senescence

In [ ]:
for i in range(len(farm)):
    x,y = farm.centroid[i].x , farm.centroid[i].y
    vis_f1[vi].sel(time=slice('2013-07','2014-04')).sel(longitude = x, latitude = y, method='nearest').plot();plt.title(i);plt.grid();plt.show();plt.close()

In [ ]:
# the metrics
basic_pheno_stats = [
    "SOS","vSOS",
    "POS", "vPOS",
    "EOS", "vEOS",
    "Trough", "LOS","AOS",
    "ROG","ROS",
]
method_sos = "first"
method_eos = "last"

#### filtering months
https://stackoverflow.com/questions/52533630/how-to-select-an-inter-year-period-with-xarray

select only daily data from June, July, and August

da_jja_only = ds.sel(time=ds.time.dt.month.isin([6, 7, 8]))

### gotta fix the YEAR -> to SEASON 
the way the original code is written is for the northern hemisphere, not here!

In [ ]:
vis_jd = vis_f1.sel(time=vis_f1.time.dt.month.isin([6,7,8,9,10,11,12,1,2]))

#### teste safrinha

In [ ]:
safrinha = True
if safrinha:
    vis_sel = vis_f1.sel(time=vis_f1.time.dt.month.isin([2,3,4,5,6,7,8]))

In [ ]:
safra = True
anos = np.unique(vis_f1.time.dt.year)
anos

In [ ]:
ano = 2013
da = vis_f1[vi].sel(time=slice(str(ano)+'-06-20',str(ano+1)+'-03-01'))
da

In [ ]:
safrinha = False
safra = True

# store results in dict
pheno_results = {}
# get years
if safrinha:
    years = vis_sel[vi].groupby("time.year") #FIXME acho que a confusão está aqui
    years_int = [y[0] for y in years]

    for year in years_int:
        da = dict(years)[year]
            # calculate stats
        stats = xr_phenology( da, method_sos=method_sos, method_eos=method_eos, stats=basic_pheno_stats,
                            verbose=False, )
        # add results to dict
        pheno_results[str(year)] = stats

if safra:
    years_int = np.unique(vis_f1.time.dt.year)
    for year in years_int[:-1]:
        da = vis_f1[vi].sel(time=slice(str(year)+'-06-20',str(year+1)+'-03-01'))
            # calculate stats
        stats = xr_phenology( da, method_sos=method_sos, method_eos=method_eos, stats=basic_pheno_stats,
                            verbose=False, )
        # add results to dict
        pheno_results[str(year)] = stats


# # loop through years and calculate phenology
# for year in years_int:
#     if safrinha:
#         # select year
#         da = dict(years)[year]
#     if safra:
#         da = vis_f1[vi].sel(time=slice(str(year)+'-06-20',str(year+1)+'-03-01'))

#     # calculate stats
#     stats = xr_phenology(
#         da,
#         method_sos=method_sos,
#         method_eos=method_eos,
#         stats=basic_pheno_stats,
#         verbose=False,
#     )
#     # add results to dict
#     pheno_results[str(year)] = stats

#### data

In [ ]:
for year in years_int[:-1]:
    year = str(year)
    pheno_results[year].to_netcdf(f'{folder}/Pheno_LAI{year}_safra.nc')

#### plots

In [ ]:
# PLOT Phenology
def plot_pheno( pheno_results, year, pheno, farm, parcels = 'None'):

    da = pheno_results[str(year)][pheno]
    if pheno in ['vSOS', 'vPOS', 'vEOS', 'AOS', 'Trough', 'LOS']:
        cmap = 'RdYlBu'
    if pheno in ['SOS', 'POS', 'EOS']:
        cmap = 'bwr'
    if pheno in ['ROG' , 'ROS']:
        cmap = 'magma'
    

    plotao = da.hvplot.quadmesh(x = 'longitude' , y = 'latitude', 
                geo=True, cmap=cmap, robust = True,  height = 400, 
                width = 600) * parcels.hvplot(geo=True,  line_width = 2, 
                                                line_color = 'green',  color='none') * farm.hvplot(geo=True, 
                                                                            line_width = 3, line_color = 'black', color='none') 
    plotao.opts(opts.Overlay(title=pheno))
    # if parcels == 'None':
    #     plotao = da.hvplot.quadmesh(x = 'longitude' , y = 'latitude', 
    #                 geo=True, cmap=cmap, robust = True,  height = 400, 
    #                 width = 600) * farm.hvplot(geo=True, line_width = 3, line_color = 'black', color='none') 
    #     plotao.opts(opts.Overlay(title=pheno))
    
    
    return plotao

In [ ]:
y = '2020'
phenos = ['SOS', 'POS', 'EOS','vSOS', 'vPOS', 'vEOS', 'AOS', 'Trough', 'LOS']
plot_pheno(pheno_results=pheno_results, year=y, pheno='AOS', farm=farm, parcels=parcels)

In [ ]:
plot_pheno(pheno_results=pheno_results, year=y, pheno='SOS', farm=farm, parcels=parcels)

In [ ]:
plot_pheno(pheno_results=pheno_results, year=y, pheno='vSOS', farm=farm, parcels=parcels)

In [ ]:
plot_pheno(pheno_results=pheno_results, year=y, pheno='POS', farm=farm, parcels=parcels)

In [ ]:
plot_pheno(pheno_results=pheno_results, year=y, pheno='vPOS', farm=farm, parcels=parcels)

In [ ]:
plot_pheno(pheno_results=pheno_results, year=y, pheno='EOS', farm=farm, parcels=parcels)

In [ ]:
plot_pheno(pheno_results=pheno_results, year=y, pheno='vEOS', farm=farm, parcels=parcels)

In [ ]:
plot_pheno(pheno_results=pheno_results, year=y, pheno='LOS', farm=farm, parcels=parcels)

In [ ]:
plot_pheno(pheno_results=pheno_results, year=y, pheno='Trough', farm=farm, parcels=parcels)

In [ ]:
plot_pheno(pheno_results=pheno_results, year=y, pheno='ROG', farm=farm, parcels=parcels)

In [ ]:
plot_pheno(pheno_results=pheno_results, year=y, pheno='ROS', farm=farm, parcels=parcels)

### The matplotlib way

In [ ]:
da = pheno_results['2013']['AOS']

fig, ax = plt.subplots(figsize=(15,8))
da.plot.pcolormesh(x = 'longitude' , y = 'latitude', ax=ax, cmap='RdYlBu')
farm.plot(ax=ax, edgecolor='black', lw = 3, color='none')